In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier

import time

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',
            # 'Macro AUC': '',
            # 'Weighted AUC': '',
            'TPR': '',
            'FNR': '',
            'TNR': '',
            'FPR': '',
        }
head = pd.DataFrame([head])
head.to_csv("/home/jovyan/UNSW/transfer_attack/attackmodel.csv", mode='a', index=False)




In [3]:
def calculate_performance_metrics(X_test, y_true, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_true))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
        # macro_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='macro')
        # weighted_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='weighted')

        cm = confusion_matrix(all_labels, all_preds)

        def calculate_class_metrics_macro(cm, class_index):
            TP = cm[class_index, class_index]
            FP = cm[:, class_index].sum() - TP
            FN = cm[class_index, :].sum() - TP
            TN = cm.sum() - (TP + FP + FN)
            
            TPR = TP / (TP + FN) if (TP + FN) != 0 else 0  
            TNR = TN / (TN + FP) if (TN + FP) != 0 else 0  
            FPR = FP / (FP + TN) if (FP + TN) != 0 else 0  
            FNR = FN / (FN + TP) if (FN + TP) != 0 else 0  
            
            return TPR, TNR, FPR, FNR
            
        metrics = np.array([calculate_class_metrics_macro(cm, i) for i in range(num_classes)])
        TPR_macro, TNR_macro, FPR_macro, FNR_macro = np.mean(metrics, axis=0)

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        
        print(f"Mean FNR: {FNR_macro}\nMean TNR: {TNR_macro}\nMean FPR: {FPR_macro}\nMean TPR: {TPR_macro}")

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,
            # 'Macro AUC': macro_auc,
            # 'Weighted AUC': weighted_auc,
            'TPR': TPR_macro,
            'FNR': FNR_macro,
            'TNR': TNR_macro,
            'FPR': FPR_macro,
        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("/home/jovyan/UNSW/transfer_attack/attackmodel.csv", mode='a', index=False, header=False)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [5]:
x_train = np.load("/home/jovyan/UNSW/x_train.npy")
y_train = np.load("/home/jovyan/UNSW/y_train.npy")
x_val = np.load("/home/jovyan/UNSW/x_val.npy")
y_val = np.load("/home/jovyan/UNSW/y_val.npy")
x_test = np.load("/home/jovyan/UNSW/x_test.npy")
y_test = np.load("/home/jovyan/UNSW/y_test.npy")

In [6]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [7]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [8]:
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
model.load_state_dict(torch.load("/home/jovyan/UNSW/transfer_attack/dnn_pytorch.pt"))

<All keys matched successfully>

In [9]:
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
    
classifier = PyTorchClassifier(
    model=model,
    clip_values=(-5, 5),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(input_shape,),
    nb_classes=output_shape,
    device_type='gpu'
)

In [11]:
from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent


In [ ]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
for epsilon in epsilon_values:
    adv_crafter = BasicIterativeMethod(classifier, eps=epsilon, max_iter = 10, verbose=False) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [ ]:
for epsilon in epsilon_values:
    adv_crafter = FastGradientMethod(classifier, eps=epsilon) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [ ]:
for epsilon in epsilon_values:
    adv_crafter = ProjectedGradientDescent(classifier, eps=epsilon, verbose=False, max_iter=10) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [13]:
# DF_eps = [1e-07, 1e-06, 1e-05, 1e-04]
epsilon_values = [0.01, 0.1, 0.2, 0.3]

for epsilon in epsilon_values:
    adv_crafter = DeepFool(classifier, epsilon=epsilon, verbose=False, max_iter=10) # init eps=1e-06
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)


In [ ]:
from art.attacks.evasion import AutoProjectedGradientDescent

for epsilon in epsilon_values:
    adv_crafter = AutoProjectedGradientDescent(classifier, eps=epsilon, max_iter = 10, batch_size=128, verbose = False) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_AutoPGD_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [22]:
from art.attacks.evasion import ZooAttack

ZOO_epsilon_values = [0, 0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
for epsilon in ZOO_epsilon_values:
    adv_crafter = ZooAttack(classifier, confidence=epsilon, max_iter = 3, batch_size=1, nb_parallel=20) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_ZOO_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

ZOO: 100%|██████████| 113971/113971 [1:10:41<00:00, 26.87it/s]


In [17]:
x_test.shape

(113971, 56)

In [23]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
for epsilon in epsilon_values:
    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

for epsilon in epsilon_values:
    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'FGSM', epsilon)

for epsilon in epsilon_values:
    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'PGD', epsilon)

# DF_eps = [1e-07, 1e-06, 1e-05, 1e-04]
for epsilon in epsilon_values:
    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'DF', epsilon)

for epsilon in epsilon_values:
    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_AutoPGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'AutoPGD', epsilon)

Accuracy: 0.843828693264076

macro
Precision: 0.8287060936095525
Recall: 0.8740235190065506
F1 Score: 0.8349328751791423
AUC: 0.9604119816641572

weighted
Precision: 0.8817169535819992
Recall: 0.843828693264076
F1 Score: 0.8487755589184576
AUC: 0.9604119816641572

Mean FNR: 0.12597648099344932
Mean TNR: 0.8740235190065506
Mean FPR: 0.12597648099344932
Mean TPR: 0.8740235190065506
Accuracy: 0.43225908345105335

macro
Precision: 0.4557881192711336
Recall: 0.4500755206750682
F1 Score: 0.42488081695346236
AUC: 0.45231104885690687

weighted
Precision: 0.5188841873847614
Recall: 0.43225908345105335
F1 Score: 0.44841250879421807
AUC: 0.45231104885690687

Mean FNR: 0.5499244793249318
Mean TNR: 0.4500755206750682
Mean FPR: 0.5499244793249318
Mean TPR: 0.4500755206750682
Accuracy: 0.028112414561599002

macro
Precision: 0.02169078753942103
Recall: 0.042531006156571766
F1 Score: 0.027526297223255683
AUC: 0.004472184716702642

weighted
Precision: 0.016037553290269083
Recall: 0.028112414561599002
F1

In [15]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

for epsilon in epsilon_values:
    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'DF', epsilon)

for epsilon in epsilon_values:
    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_AutoPGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'AutoPGD', epsilon)

Accuracy: 0.10088531293048232

macro
Precision: 0.11750893692137852
Recall: 0.1130588981154592
F1 Score: 0.10083944624702498

weighted
Precision: 0.13481696168756185
Recall: 0.10088531293048232
F1 Score: 0.10315931998734723

Mean FNR: 0.8869411018845408
Mean TNR: 0.1130588981154592
Mean FPR: 0.8869411018845408
Mean TPR: 0.1130588981154592
Accuracy: 0.11687183581788349

macro
Precision: 0.13796072207534155
Recall: 0.1242853482615926
F1 Score: 0.11634683787586446

weighted
Precision: 0.16233951011706008
Recall: 0.11687183581788349
F1 Score: 0.12412750392009254

Mean FNR: 0.8757146517384073
Mean TNR: 0.1242853482615926
Mean FPR: 0.8757146517384073
Mean TPR: 0.1242853482615926
Accuracy: 0.12687438032481946

macro
Precision: 0.1498148345125979
Recall: 0.13101375790289818
F1 Score: 0.12574959692608464

weighted
Precision: 0.1784398213095376
Recall: 0.12687438032481946
F1 Score: 0.13707749744206096

Mean FNR: 0.8689862420971018
Mean TNR: 0.13101375790289818
Mean FPR: 0.8689862420971018
Mean T

In [24]:
print("start ZOO")
ZOO_epsilon_values = [0, 0.01, 0.1, 0.2, 0.3]
# Iterate over epsilon values
for epsilon in ZOO_epsilon_values:
    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_ZOO_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'ZOO', epsilon)

start ZOO
Accuracy: 0.8617630800817752

macro
Precision: 0.849058400263804
Recall: 0.8260089098098148
F1 Score: 0.8358429550842166

weighted
Precision: 0.8596456763044009
Recall: 0.8617630800817752
F1 Score: 0.8594067571128883

Mean FNR: 0.17399109019018522
Mean TNR: 0.8260089098098148
Mean FPR: 0.17399109019018522
Mean TPR: 0.8260089098098148
Accuracy: 0.8622368848215775

macro
Precision: 0.8497054803530435
Recall: 0.8264663363701825
F1 Score: 0.8363730738482078

weighted
Precision: 0.8601382236319954
Recall: 0.8622368848215775
F1 Score: 0.8598732274848737

Mean FNR: 0.17353366362981748
Mean TNR: 0.8264663363701825
Mean FPR: 0.17353366362981748
Mean TPR: 0.8264663363701825
Accuracy: 0.8618069508910161

macro
Precision: 0.8491943563731615
Recall: 0.8259463611217005
F1 Score: 0.8358529013267217

weighted
Precision: 0.8596917208564451
Recall: 0.8618069508910161
F1 Score: 0.8594314042639348

Mean FNR: 0.17405363887829953
Mean TNR: 0.8259463611217005
Mean FPR: 0.17405363887829953
Mean TPR: